# Initial Ideas for evaporator code

Here's some simple ideas for how best to code an evaporating body

#### Physical approach

Let's think about the real physics going on, and see if we can adapt known models to account for that.

The cause of the signal in evaporator transits is not the planet, but the dust cloud.

To me, this dust cloud has three possibilities:
 * Spherical cloud around planet which decreases density with distance
 * Smeared tail behind planet which decreases density with distance
 * Smeared tail in front of planet which decreases density with distance
 
In all three cases, we could iteractively fit a Mandel+Agol transit model to account for the change in size over time. e.g:
 
 <img src="MandelAgolIdeas.png">
 
We would then have a couple of hyper parameters for each transit governing:
* the direction of motion of the dust cloud w.r.t. the planet (+ being leading, 0 being sphere, - being trailing)
* the rate of dispersion over time
* total dust production

In this sort of model, the dust would be conserved - i.e. the density of the dust between transit model iterations would be proportional to the change in area (i.e. dispersion)

##### Forward-scattering

